The process to authenticate Earth Engine

In [4]:
"""This script is used to download satelites images from Google Earth Engine."""
import os
import zipfile
import ee
import requests
from PIL import Image

# Initializes the import to get functionality
ee.Initialize()
# ee.Authenticate()


In [5]:
#RnkUmmi -52.89 71.55 -51.43 71.82
#Creates a geometry object with cordinates of RnkUmmi
geometry = ee.Geometry.Rectangle([-52.89, 71.55, -51.43, 71.82])

DATA_PRODUCT = 'LANDSAT/LE07/C01/T1'
SEL_BANDS = ['B1','B2','B3']

#current dir
CURRENT_DIR = os.getcwd()

#creates a temp folder to download the images to
SAVE_FOLDER = os.path.join(CURRENT_DIR,'RnkUmmi_Landsat-7')

if not os.path.exists(SAVE_FOLDER):
    # Create the directory if it doesn't exist
    os.makedirs(SAVE_FOLDER)

frames = []

In [6]:
#For loops that traverses all the years sequentially
for year in range(1999,2013):
    start = ee.Date(f'{year}-07-20')
    end = ee.Date(f'{year}-09-10')

    #This is a imageCollection object that has all the filters applied
    #Creates a new collection everytime there is a new year
    filteredCollection = (
        ee.ImageCollection(DATA_PRODUCT)
        .filterBounds(geometry)
        .filterDate(start, end)
        .select(SEL_BANDS)
        .filterMetadata('CLOUD_COVER', 'less_than', 30)
    )

    #Why is this list created? Its an object that holds pictures. Just need the pictures
    filtered_list=filteredCollection.toList(filteredCollection.size())

    #gets the count of the list
    filteredCount = filtered_list.size().getInfo()
    print(f"Filtered count is {filteredCount}")
    print(
    f"year is {year}, "
    f"{start.format('yyyy-MM-dd').getInfo()}" 
    f" and {end.format('yyyy-MM-dd').getInfo()}"
    )
    # for loop that traveses all the pictures in the list
    for i in range(filteredCount):
        #Make sure the list are inside the geometry object
        image = ee.Image(filtered_list.get(i)).clip(geometry)
        #Prints and formats the name of the GeoTIFF that is being downloaded
        fname = image.get('system:id').getInfo().split('/')[-1]

        # image = combine_bands(image)
        print(f"Downloading {fname}")
        try:
            #creates the download url for the image
            url = image.getDownloadURL()
            #makes a request to download the image
            r=requests.get(url)
        except ConnectionError as e:
            print(f"Connection error: {e}")
        except TimeoutError as e:
            print(f"Time out error: {e}")
        except Exception as e:
            print(f"Failed to get Download URL: {e}")

        #names each file
        filename = f"data_{i}.zip"
        filepath = os.path.join(SAVE_FOLDER, filename)
        print(f"Downloading to: {filepath}, {i} in Filtered Count")
        try:
            #Downloads zip
            with open(filepath, 'wb') as f:
                f.write(r.content)
        except Exception as e:
            print("Error failure to download")
        #Extract zip
        try:
            with zipfile.ZipFile(filepath) as f:
                files = f.namelist()
                f.extractall(SAVE_FOLDER)
                imgb = Image.open(f"{SAVE_FOLDER}/{files[0]}")
                imgg = Image.open(f"{SAVE_FOLDER}/{files[1]}")
                imgr = Image.open(f"{SAVE_FOLDER}/{files[2]}")
                true_color_image = Image.merge("RGB", (imgr, imgg, imgb))
                if not os.path.exists(f"{SAVE_FOLDER}/true_color"):
                    os.makedirs(f"{SAVE_FOLDER}/true_color")
                true_color_image.save(f"{SAVE_FOLDER}/true_color/{fname}.tiff",'TIFF')
                frames.append(true_color_image)

                os.remove(filepath)
            print(f"Downloaded {filename}")
            print(f'Extracted {files} tif files from {filename}')
        except zipfile.BadZipFile:
            print('Skipping - Not a valid ZIP file')
        except Exception as e:
            print('Error extracting')

print("Downloading complete")


Filtered count is 4
year is 1999, 1999-07-20 and 1999-09-10
Downloaded data_0.zip
Extracted ['LE07_011010_19990801.B1.tif', 'LE07_011010_19990801.B2.tif', 'LE07_011010_19990801.B3.tif'] tif files from data_0.zip
Downloaded data_1.zip
Extracted ['LE07_012009_19990824.B1.tif', 'LE07_012009_19990824.B2.tif', 'LE07_012009_19990824.B3.tif'] tif files from data_1.zip
Downloaded data_2.zip
Extracted ['LE07_012010_19990824.B1.tif', 'LE07_012010_19990824.B2.tif', 'LE07_012010_19990824.B3.tif'] tif files from data_2.zip
Downloaded data_3.zip
Extracted ['LE07_013009_19990831.B1.tif', 'LE07_013009_19990831.B2.tif', 'LE07_013009_19990831.B3.tif'] tif files from data_3.zip
Filtered count is 5
year is 2000, 2000-07-20 and 2000-09-10
Downloaded data_0.zip
Extracted ['LE07_012009_20000810.B1.tif', 'LE07_012009_20000810.B2.tif', 'LE07_012009_20000810.B3.tif'] tif files from data_0.zip
Downloaded data_1.zip
Extracted ['LE07_012009_20000826.B1.tif', 'LE07_012009_20000826.B2.tif', 'LE07_012009_20000826.B3.

TypeError: close() missing required argument 'fd' (pos 1)

To create the gif

In [11]:
gif = frames[0].save(f'{SAVE_FOLDER}/testgif.gif',
               save_all=True,
               append_images=frames[1:],
               optimize=False,
               duration=1000,
               loop=0)  # loop=0 for infinite loop
